In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer
from sklearn.decomposition import PCA

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
datos_combi  = pd.read_csv(path_archivo)

In [3]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [4]:
dat_copy = datos_combi.copy()
N_eig = 20
target = "phi_K"
features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
features_dim = ["eta", "beta"]
feature_especial = ["eig_0"]
features_tot = [target] + feature_especial + features_dim + features_eig
for i in range(10):
    key_mod = "eig_" + str(i+1)
    prev_key = "eig_" + str(i)
    if i == 0:  
        dat_copy[key_mod] = 1/dat_copy[key_mod]
    else:
        dat_copy[key_mod] = dat_copy[prev_key]/dat_copy[key_mod]
    #fin if 
#fin for
datos_combi = dat_copy[features_tot].copy()


In [5]:
datos_combi.head()

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_11,eig_12,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19,eig_20
0,0.002000,3.910766e-07,0.19635,1.047198,0.123874,0.009468,0.000346,0.00001,1.099366e-07,9.328398e-10,...,1164.932469,1171.482800,1449.038005,2300.955385,2502.368311,3312.025456,5345.565470,5997.709361,10470.224168,10740.077078
1,0.005138,1.003534e-06,0.19635,1.047198,0.123881,0.009468,0.000346,0.00001,1.098275e-07,9.309612e-10,...,1166.101326,1171.591939,1449.122420,2303.158531,2503.364568,3312.436183,5348.557329,5999.020979,10453.936163,10723.558723
2,0.008275,1.614694e-06,0.19635,1.047198,0.123887,0.009468,0.000345,0.00001,1.097185e-07,9.290879e-10,...,1167.270192,1171.700766,1449.206420,2305.361662,2504.359361,3312.844974,5351.545841,6000.326889,10437.784785,10707.179166
3,0.011413,2.224556e-06,0.19635,1.047198,0.123893,0.009468,0.000345,0.00001,1.096098e-07,9.272199e-10,...,1168.439095,1171.809289,1449.290016,2307.564833,2505.352724,3313.251862,5354.531096,6001.627165,10421.768356,10690.936703
4,0.014550,2.833116e-06,0.19635,1.047198,0.123899,0.009468,0.000344,0.00001,1.095012e-07,9.253570e-10,...,1169.608057,1171.917510,1449.373208,2309.768079,2506.344673,3313.656851,5357.513142,6002.921832,10405.885145,10674.829574


In [6]:
for i in range(N_eig):
    col = "eig_" + str(i+1)
    datos_combi[col] = np.log(datos_combi[col])
#fin for 
datos_combi = datos_combi.copy()
datos_combi.head()

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_11,eig_12,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19,eig_20
0,0.002000,3.910766e-07,0.19635,1.047198,-2.088487,-4.659885,-7.969479,-11.478219,-16.023362,-20.792788,...,7.060418,7.066026,7.278655,7.741080,7.824993,8.105315,8.584023,8.699133,9.256291,9.281738
1,0.005138,1.003534e-06,0.19635,1.047198,-2.088436,-4.659868,-7.970508,-11.479121,-16.024355,-20.794803,...,7.061421,7.066119,7.278713,7.742037,7.825391,8.105439,8.584582,8.699352,9.254734,9.280198
2,0.008275,1.614694e-06,0.19635,1.047198,-2.088385,-4.659852,-7.971537,-11.480022,-16.025348,-20.796818,...,7.062423,7.066212,7.278771,7.742993,7.825788,8.105563,8.585141,8.699569,9.253188,9.278670
3,0.011413,2.224556e-06,0.19635,1.047198,-2.088335,-4.659835,-7.972564,-11.480924,-16.026339,-20.798830,...,7.063424,7.066304,7.278829,7.743948,7.826185,8.105685,8.585698,8.699786,9.251652,9.277152
4,0.014550,2.833116e-06,0.19635,1.047198,-2.088285,-4.659819,-7.973591,-11.481824,-16.027331,-20.800841,...,7.064424,7.066397,7.278886,7.744902,7.826581,8.105808,8.586255,8.700002,9.250127,9.275644


In [7]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [8]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "pca" + str(x), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred_lineal/"
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [9]:
def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/", nombre_feat = "l_moño"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función

In [10]:
n_comp = 3
datos_copy = datos_combi.copy()
features = features_dim + features_eig
pca_obj = PCA(n_components = n_comp)
pca_obj.set_output(transform="pandas")
X_eigs = datos_copy[features_eig]
pca_obj.fit(X_eigs)
X_pcas = pca_obj.transform(X_eigs)
X_dims = datos_copy[features_dim]
X_train = pd.concat((X_dims, X_pcas), axis = 1)
y_train = datos_copy[target]


In [11]:
datos_full = pd.concat((X_train, y_train), axis=1)
X_train.head()

,eta,beta,pca0,pca1,pca2
0,0.19635,1.047198,-42.909317,0.487427,-0.757292
1,0.19635,1.047198,-42.915680,0.485792,-0.755665
2,0.19635,1.047198,-42.922039,0.484162,-0.754045
3,0.19635,1.047198,-42.928394,0.482537,-0.752430
4,0.19635,1.047198,-42.934745,0.480917,-0.750823


In [12]:
graficar_todas(datos_full, n_comp, target, nombre = "PCA/", nombre_feat="pcas")

In [13]:
pca_obj.components_

array([[ 0.02572875,  0.06278715,  0.10067357,  0.14625964,  0.20004316,
         0.26020569,  0.32351569,  0.39061299,  0.46376833,  0.54093503,
        -0.08361991, -0.08640764, -0.08923923, -0.09227729, -0.09410262,
        -0.09700857, -0.09917278, -0.10056593, -0.10431291, -0.10572889],
       [ 0.16630307,  0.24591815,  0.28246457,  0.29826249,  0.29728986,
         0.26440013,  0.18904839,  0.10122358, -0.02370159, -0.17496805,
         0.20023544,  0.20323675,  0.20584688,  0.22881646,  0.23114688,
         0.23104829,  0.22777043,  0.22994784,  0.24591506,  0.24392606],
       [ 0.06246792,  0.32524815,  0.30077331,  0.2809703 ,  0.26205981,
         0.21159646, -0.01563057, -0.19130138, -0.28807186, -0.31653639,
        -0.08541228, -0.12648403, -0.15275743, -0.20006331, -0.1886075 ,
        -0.18439057, -0.19143177, -0.22206913, -0.26543575, -0.27347692]])

In [14]:
test1 = np.array(pca_obj.components_).dot(X_eigs.iloc[0])